In [1]:
import pandas as pd
import numpy as np
import h5py
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Input
from sklearn.preprocessing import MinMaxScaler

In [2]:
train_data = pd.read_csv('heart_rate_data_train.csv')
test_data = pd.read_csv('heart_rate_data_test.csv')
print(train_data.dtypes)

uuid          object
SD1          float64
SD2          float64
sampen       float64
higuci       float64
datasetId      int64
condition     object
dtype: object


In [3]:
train_data = train_data.select_dtypes(include=[np.number])
test_data = test_data.select_dtypes(include=[np.number])

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)

In [5]:
def preprocess_data(data, time_steps):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i, :-1])  # Use all columns except the last as features
        y.append(data[i, -1])  # Use the last column as the label (heart rate)
    return np.array(X), np.array(y)

In [6]:
time_steps = 10

# Preprocess the data
X_train, y_train = preprocess_data(train_data_scaled, time_steps)
X_test, y_test = preprocess_data(test_data_scaled, time_steps)

In [7]:
model = Sequential()

# Define the input shape with an Input layer
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

# Add LSTM layers
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 10, 50)              │          11,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 10, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,251 (122.07 KB)

 Trainable params: 31,251 (122.07 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
11540/11540 ━━━━━━━━━━━━━━━━━━━━ 193s 16ms/step - loss: 4.8513e-05 - val_loss: 1.0757e-09
Epoch 2/10
11540/11540 ━━━━━━━━━━━━━━━━━━━━ 184s 16ms/step - loss: 1.3465e-08 - val_loss: 1.1534e-09
Epoch 3/10
11540/11540 ━━━━━━━━━━━━━━━━━━━━ 172s 15ms/step - loss: 7.2296e-10 - val_loss: 1.2181e-09
Epoch 4/10
11540/11540 ━━━━━━━━━━━━━━━━━━━━ 157s 14ms/step - loss: 7.2695e-10 - val_loss: 1.6930e-11
Epoch 5/10
11540/11540 ━━━━━━━━━━━━━━━━━━━━ 163s 14ms/step - loss: 7.4273e-10 - val_loss: 4.8000e-11
Epoch 6/10
11540/11540 ━━━━━━━━━━━━━━━━━━━━ 173s 15ms/step - loss: 7.2344e-10 - val_loss: 6.0466e-12
Epoch 7/10
11540/11540 ━━━━━━━━━━━━━━━━━━━━ 179s 16ms/step - loss: 7.1758e-10 - val_loss: 4.0668e-10
Epoch 8/10
11540/11540 ━━━━━━━━━━━━━━━━━━━━ 133s 11ms/step - loss: 7.2011e-10 - val_loss: 1.0122e-10
Epoch 9/10
11540/11540 ━━━━━━━━━━━━━━━━━━━━ 99s 9ms/step - loss: 7.2045e-10 - val_loss: 1.4668e-10
Epoch 10/10
11540/11540 ━━━━━━━━━━━━━━━━━━━━ 189s 16ms/step - loss: 7.3114e-10 - val_loss: 8.

In [9]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [17]:
model.save('DHT.keras')